In [ ]:
import os
import tensorflow as tf
print(tf.__version__)

In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import os, sys, glob, csv, keras
from sklearn import model_selection, preprocessing
from os import walk, path
from keras import models, layers, optimizers, preprocessing as KRSpreps, utils as KRSutils
#from tslearn import preprocessing as TSpreps, utils as TSutils
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.svm import SVC

In [ ]:
from __future__ import absolute_import, division, print_function

from tensorflow.keras.layers import Lambda, Input, Dense, Flatten, Dropout, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.losses import mse, binary_crossentropy
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K

#import numpy as np, matplotlib.pyplot as plt, argparse, os

In [ ]:
from tensorflow import keras
import tempfile

import matplotlib as mpl
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from glob import glob
import datetime

### Create Dictionary and store paths for all different Modalities(Gaze)

In [ ]:
data_path = {}
data_path['gazedata_path'] = "Gaze_Features/"

### Checking No. of files in each of Gaze Folders && Shape of the Dataframes

In [ ]:
os.chdir("/Users/jingweiong/Downloads/Deception-Detection-master")

In [ ]:
dir = "/Users/jingweiong/Downloads/Deception_detection_output"
key = data_path['gazedata_path'].split('_')[0]
data_shape_all = pd.DataFrame()
count = 0
data_shape, file_names = [], []
for filepath in glob(path.join(dir, '*.csv')):
    file_shape = pd.read_csv(filepath).shape
    filename = path.basename(filepath)
    if filename not in ['Annotation_gaze_features.csv']:
        data_shape.append([file_shape[0], file_shape[1]])
        file_names.append(filename)
        count += 1
data_shape = pd.DataFrame(data_shape)
data_shape.columns = [key + str(0), key + str(1)]
data_shape.index = pd.Series(file_names)
data_shape_all = pd.concat([data_shape_all, data_shape], axis=1, sort=True)
print(f"No. of file in {key}: ", count)

print(data_shape_all)

## Creating Dictionaries of Gaze
Remove Initials and Make the Keys Same for the Same data

In [ ]:
gaze_dict = {}

for filepath in glob(path.join(dir, '*.csv')):
    data = pd.read_csv(filepath)
    filename = path.basename(filepath)
    filename = filename.replace("Gaze_", "")
    gaze_dict[filename] = data

print(gaze_dict)

In [ ]:
filename_dictkeys = list(gaze_dict)
label_dict = {}

for key in filename_dictkeys:
    gazedata = gaze_dict[key]
    label_gaze = gazedata.loc[:, "label"].unique()[0]
    label_dict[key] = label_gaze

print(label_dict)

In [ ]:
from scipy.signal import resample

filename_dictkeys = list(gaze_dict)
gaze_dict_upsampled = {}

for key in filename_dictkeys:
    gaze_data = gaze_dict[key]
    if "Unnamed: 0" in gaze_data.columns:
        gaze_data = gaze_data.drop("Unnamed: 0", axis=1)
    else:
        print(f"Warning: 'Unnamed: 0' column not found in '{key}'")
    
    # Drop other columns like "frame" and "label"
    gaze_data = gaze_data.drop(["frame", "label", "face_id", "timestamp", "confidence", "success"], axis=1, errors='ignore')

    gaze_data = np.array(gaze_data.drop_duplicates())
    gaze_dict_upsampled[key] = resample(gaze_data, 300)

print(gaze_dict_upsampled)

### Splitting Train and Test data for Gaze, Audio, Micro-expression data

In [ ]:
import random

random.seed(3986)

# Splitting keys for train and test sets
train_split_keys = random.sample(list(gaze_dict_upsampled), int(0.75 * len(list(gaze_dict_upsampled))))
test_split_keys = list(set(list(gaze_dict_upsampled)) - set(train_split_keys))

dict_split_train = lambda dict_data: {key: value for key, value in dict_data.items() if key in train_split_keys}
dict_split_test = lambda dict_data: {key: value for key, value in dict_data.items() if key in test_split_keys}

mapped_train = list(map(dict_split_train, [gaze_dict_upsampled, label_dict]))
mapped_test = list(map(dict_split_test, [gaze_dict_upsampled, label_dict]))

train_data = {'gaze_train': mapped_train[0], 'y_train': mapped_train[1]}
test_data = {'gaze_test': mapped_test[0], 'y_test': mapped_test[1]}
print(train_data)   

In [ ]:
conv2arr = lambda x: np.array(x)

train_data['gaze_train'] = conv2arr(list(train_data['gaze_train'].values()))
test_data['gaze_test'] = conv2arr(list(test_data['gaze_test'].values()))

train_data['y_train'] = conv2arr(list(train_data['y_train'].values()))
test_data['y_test'] = conv2arr(list(test_data['y_test'].values()))

### Removing Ground Truth Values from && scaling data

In [ ]:
from sklearn.preprocessing import StandardScaler

# List of keys excluding label keys
train_key, test_key = list(train_data.keys()), list(test_data.keys())
train_key.remove('y_train')
test_key.remove('y_test')

for key1, key2 in zip(train_key, test_key):
    scaler = StandardScaler()
    data1, data2 = train_data[key1], test_data[key2]

    s0, s1, s2 = data1.shape[0], data1.shape[1], data1.shape[2]
    print(data1.shape)
    data1 = data1.reshape(s0*s1, s2)
    data1 = scaler.fit_transform(data1)
    print(data1.shape)
    train_data[key1] = data1.reshape(s0, s1, s2)

    s0, s1, s2 = data2.shape[0], data2.shape[1], data2.shape[2]
    print(data2.shape)
    data2 = data2.reshape(s0*s1, s2)
    data2 = scaler.transform(data2)
    print(data2.shape)
    test_data[key2] = data2.reshape(s0, s1, s2)
    print(data1.shape, data2.shape)

### Finally printing the shape of data(train and test) from different modalities

In [ ]:
gaze_features = train_data['gaze_train'].copy()
test_gaze_features = test_data['gaze_test'].copy()

gaze_features = np.clip(gaze_features, -5, 5)
test_gaze_features = np.clip(test_gaze_features, -5, 5)

print('Gaze train shape:', gaze_features.shape)
print('Gaze test shape:', test_gaze_features.shape)

### Encoding Truthful and deceptive ground truth values

In [ ]:
y_train = pd.Series(train_data['y_train']).apply(lambda value: 0 if value == 'Truthful' else 1)
y_test = pd.Series(test_data['y_test']).apply(lambda value: 0 if value == 'Truthful' else 1)

In [ ]:
# Reshape gaze_features to a 2D array
num_samples, time_steps, num_features = gaze_features.shape
flattened_features = gaze_features.reshape(num_samples, time_steps * num_features)

# Create SVM classifier with a linear kernel
svm_classifier = SVC(kernel='linear')

# Check for NaN values
nan_indices = np.isnan(flattened_features)
if np.any(nan_indices):
    # Handle NaN values (e.g., impute or remove)
    # For example, you can replace NaN with the mean of each feature
    mean_features = np.nanmean(flattened_features, axis=0)
    flattened_features[nan_indices] = np.take(mean_features, np.nonzero(nan_indices)[1])

# Train the SVM model
svm_classifier.fit(flattened_features, y_train)

In [ ]:
# Reshape test_gaze_features to a 2D array
num_samples_test, time_steps_test, num_features_test = test_gaze_features.shape
flattened_features_test = test_gaze_features.reshape(num_samples_test, time_steps_test * num_features_test)

# Make predictions on the test data
predictions = svm_classifier.predict(flattened_features_test)

# Evaluate the model's performance
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

In [ ]:
# Create SVM classifier with a polynomial kernel
svm_classifier = SVC(kernel='poly')

# Train the SVM model
svm_classifier.fit(flattened_features, y_train)

# Make predictions on the test data
predictions = svm_classifier.predict(flattened_features_test)

# Evaluate the model's performance
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

from sklearn.metrics import confusion_matrix

# Calculate the confusion matrix
conf_matrix = confusion_matrix(y_test, predictions)

# Print the confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
# Create SVM classifier with a rbf kernel
svm_classifier = SVC(kernel='rbf')

# Train the SVM model
svm_classifier.fit(flattened_features, y_train)

# Make predictions on the test data
predictions = svm_classifier.predict(flattened_features_test)

# Evaluate the model's performance
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

from sklearn.metrics import confusion_matrix

# Calculate the confusion matrix
conf_matrix = confusion_matrix(y_test, predictions)

# Print the confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
# Create SVM classifier with a sigmoid kernel
svm_classifier = SVC(kernel='sigmoid')

# Train the SVM model
svm_classifier.fit(flattened_features, y_train)

# Make predictions on the test data
predictions = svm_classifier.predict(flattened_features_test)

# Evaluate the model's performance
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

from sklearn.metrics import confusion_matrix

# Calculate the confusion matrix
conf_matrix = confusion_matrix(y_test, predictions)

# Print the confusion matrix
print("Confusion Matrix:")
print(conf_matrix)